In [ ]:
import h5py
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset

# Chargement des données
dataset_simple_path = 'D:/Data/data/dog_dataset_no_aug.h5'
#dataset_augmented_path = 'D:/Data/data/dog_dataset_aug_normal.h5'
#dataset_aug_lots_path = 'D:/Data/data/dog_dataset_aug_lots.h5'

# Classe personnalisée pour charger les données
class DogBreedDataset(Dataset):
    def __init__(self, dataset_simple_path, transform=None):
        self.file_path = dataset_simple_path
        self.transform = transform
        
        # Charger uniquement les dimensions
        with h5py.File(self.file_path, "r") as f:
            if "images" in f:
                self.data_len = f["images"].shape[0] # type: ignore
            else:
                raise KeyError("Dataset 'images' not found in the HDF5 file.")
        
    def __len__(self):
        return self.data_len

    def __getitem__(self, idx):
        with h5py.File(self.file_path, "r") as f:
            image = f["images"][idx] / 255.0  # type: ignore # Normalisation
            label = f["labels"][idx] # type: ignore
        
        # Appliquer une transformation éventuelle
        if self.transform:
            image = self.transform(image)

        # Format PyTorch
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
        label = torch.tensor(label, dtype=torch.long)
        
        return image, label

# Initialisation des datasets
dataset = DogBreedDataset(dataset_simple_path)

# Division en ensembles
from sklearn.model_selection import train_test_split

indices = list(range(len(dataset)))
train_indices, temp_indices = train_test_split(indices, test_size=0.3, random_state=42)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.5, random_state=42)

# Création des DataLoaders
from torch.utils.data import Subset

train_loader = DataLoader(Subset(dataset, train_indices), batch_size=32, shuffle=True)
val_loader = DataLoader(Subset(dataset, val_indices), batch_size=32, shuffle=False)
test_loader = DataLoader(Subset(dataset, test_indices), batch_size=32, shuffle=False)

Entrainement dataset aug

In [8]:
# Chargement des données
dataset_augmented_path = 'D:/ApprentissageMachineTPFinal/data/dog_dataset_aug_normal.h5'

# Classe personnalisée pour charger les données
class DogBreedDataset(Dataset):
    def __init__(self, dataset_augmented_path, transform=None):
        self.file_path = dataset_augmented_path
        self.transform = transform
        
        # Charger uniquement les dimensions
        with h5py.File(self.file_path, "r") as f:
            if "images" in f:
                self.data_len = f["images"].shape[0] # type: ignore
            else:
                raise KeyError("Dataset 'images' not found in the HDF5 file.")
        
    def __len__(self):
        return self.data_len

    def __getitem__(self, idx):
        with h5py.File(self.file_path, "r") as f:
            image = f["images"][idx] / 255.0  # type: ignore # Normalisation
            label = f["labels"][idx] # type: ignore
        
        # Appliquer une transformation éventuelle
        if self.transform:
            image = self.transform(image)

        # Format PyTorch
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
        label = torch.tensor(label, dtype=torch.long)
        
        return image, label

# Initialisation des datasets
dataset = DogBreedDataset(dataset_augmented_path)

# Division en ensembles
from sklearn.model_selection import train_test_split

indices = list(range(len(dataset)))
train_indices, temp_indices = train_test_split(indices, test_size=0.3, random_state=42)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.5, random_state=42)

# Création des DataLoaders
from torch.utils.data import Subset

train_loader = DataLoader(Subset(dataset, train_indices), batch_size=32, shuffle=True)
val_loader = DataLoader(Subset(dataset, val_indices), batch_size=32, shuffle=False)
test_loader = DataLoader(Subset(dataset, test_indices), batch_size=32, shuffle=False)

In [9]:
import torch.nn as nn
import torch.nn.functional as F

# Définition du modèle CNN
class DogBreedCNN(nn.Module):
    def __init__(self, num_classes=120):  # 120 classes de races de chiens
        super(DogBreedCNN, self).__init__()

        # couches de convolution
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)

        # Pooling et convolution
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Convolutional layers with ReLU and pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        # Flatten the output
        x = x.view(-1, 128 * 28 * 28)

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

# Initialisation du modèle
model = DogBreedCNN(num_classes=120)

Entrainement no aug

In [10]:
import torch.optim as optim

# Configurations
learning_rate = 0.001
num_epochs = 25

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Boucle d'entraînement
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

KeyboardInterrupt: 

Entrainement aug

In [ ]:
import torch.optim as optim

# Configurations
learning_rate = 0.001
num_epochs = 25

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Boucle d'entraînement
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")